### Your name:

<pre> Isaac Aktam</pre>

### Collaborators:

<pre> </pre>


In [33]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

Open the housing data


In [34]:
import os
import tarfile
from six.moves import urllib

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

fetch_housing_data()
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

housing = load_housing_data()
housing.head()





,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


### Build full pipeline for the data analysis following the example of the notebook.
 Hint: the main part requested to change is the algorithm used (Lasso regression)

If you want to learn more about the Lasso regression, see resources below:
- http://scikit-learn.org/stable/modules/linear_model.html#lasso
- https://www.analyticsvidhya.com/blog/2016/01/complete-tutorial-ridge-lasso-regression-python/

#### Considerations for building pipeline:

- Split data into training and testing sets below.
- Convert all categorical data to one-hot vectors below
- Normalize all non-categorical data 
-  Perform Lasso-based regression using a variety of values for $\alpha$ between 0 and 1 via a grid search where  *housing_labels* is the output and all other features are the input (similar to as seen in lecture two.)

In [35]:
from sklearn.linear_model import Lasso

# Write your code here:

# Step 1: Split data into training and testing sets. Use stratified sampling based on the income category.

# Add new interesting attribute combinations to work with:

housing["rooms_per_household"] = housing["total_rooms"] / housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"] / housing["total_rooms"]
housing["population_per_household"] = housing["population"] / housing["households"]

housing["income_cat"] = np.ceil(housing["median_income"] / 1.5)
housing["income_cat"].where(housing["income_cat"] < 5, 5.0, inplace = True)

# Now we are ready to do stratified sampling based on the income category.

from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2, random_state = 42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]
    
# Next, we remove the income_Cat attribute so the data is back to its original state:

for set in (strat_train_set, strat_test_set):
    set.drop(["income_cat"], axis = 1, inplace = True)
    
# Step 2, Data Cleaning

# Let's revert to a clean training set (by copying strat_train_set once again), and let's separate the predic-
# torts and the labels since we don't necessarily want to apply the same transformations to the predictors and the
# target values (not that drop() creates a copy of the data and does not affect strat_train_set):

housing = strat_train_set.drop("median_house_value", axis = 1)
housing_labels = strat_train_set["median_house_value"].copy()

# Commence data cleaning by means of Imputer. Take care of missing values.

from sklearn.preprocessing import Imputer

imputer = Imputer(strategy = "median")

# Since median can be computed can only be computed on numerical attributes, we need to create a copy of the data
# without the text attribute ocean_proximity:

housing_num = housing.drop("ocean_proximity", axis = 1)

# Now we can fit the imputer instance to the training data using the fit() method:

imputer.fit(housing_num)

# Now we can use this "trained" imputer to transform the training set by replacing missing values by the learned
# medians

X = imputer.transform(housing_num)

housing_tr = pd.DataFrame(X, columns = housing_num.columns)

# Step 3, convert all categorical data to one-hot vectors below. 

housing_cat_encoded, housing_categories = housing_cat.factorize()
housing_cat_encoded[:10]


# From here

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils import check_array
from sklearn.preprocessing import LabelEncoder
from scipy import sparse

class CategoricalEncoder(BaseEstimator, TransformerMixin):
    """Encode categorical features as a numeric array.
    The input to this transformer should be a matrix of integers or strings,
    denoting the values taken on by categorical (discrete) features.
    The features can be encoded using a one-hot aka one-of-K scheme
    (``encoding='onehot'``, the default) or converted to ordinal integers
    (``encoding='ordinal'``).
    This encoding is needed for feeding categorical data to many scikit-learn
    estimators, notably linear models and SVMs with the standard kernels.
    Read more in the :ref:`User Guide <preprocessing_categorical_features>`.
    Parameters
    ----------
    encoding : str, 'onehot', 'onehot-dense' or 'ordinal'
        The type of encoding to use (default is 'onehot'):
        - 'onehot': encode the features using a one-hot aka one-of-K scheme
          (or also called 'dummy' encoding). This creates a binary column for
          each category and returns a sparse matrix.
        - 'onehot-dense': the same as 'onehot' but returns a dense array
          instead of a sparse matrix.
        - 'ordinal': encode the features as ordinal integers. This results in
          a single column of integers (0 to n_categories - 1) per feature.
    categories : 'auto' or a list of lists/arrays of values.
        Categories (unique values) per feature:
        - 'auto' : Determine categories automatically from the training data.
        - list : ``categories[i]`` holds the categories expected in the ith
          column. The passed categories are sorted before encoding the data
          (used categories can be found in the ``categories_`` attribute).
    dtype : number type, default np.float64
        Desired dtype of output.
    handle_unknown : 'error' (default) or 'ignore'
        Whether to raise an error or ignore if a unknown categorical feature is
        present during transform (default is to raise). When this is parameter
        is set to 'ignore' and an unknown category is encountered during
        transform, the resulting one-hot encoded columns for this feature
        will be all zeros.
        Ignoring unknown categories is not supported for
        ``encoding='ordinal'``.
    Attributes
    ----------
    categories_ : list of arrays
        The categories of each feature determined during fitting. When
        categories were specified manually, this holds the sorted categories
        (in order corresponding with output of `transform`).
    Examples
    --------
    Given a dataset with three features and two samples, we let the encoder
    find the maximum value per feature and transform the data to a binary
    one-hot encoding.
    >>> from sklearn.preprocessing import CategoricalEncoder
    >>> enc = CategoricalEncoder(handle_unknown='ignore')
    >>> enc.fit([[0, 0, 3], [1, 1, 0], [0, 2, 1], [1, 0, 2]])
    ... # doctest: +ELLIPSIS
    CategoricalEncoder(categories='auto', dtype=<... 'numpy.float64'>,
              encoding='onehot', handle_unknown='ignore')
    >>> enc.transform([[0, 1, 1], [1, 0, 4]]).toarray()
    array([[ 1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.],
           [ 0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]])
    See also
    --------
    sklearn.preprocessing.OneHotEncoder : performs a one-hot encoding of
      integer ordinal features. The ``OneHotEncoder assumes`` that input
      features take on values in the range ``[0, max(feature)]`` instead of
      using the unique values.
    sklearn.feature_extraction.DictVectorizer : performs a one-hot encoding of
      dictionary items (also handles string-valued features).
    sklearn.feature_extraction.FeatureHasher : performs an approximate one-hot
      encoding of dictionary items or strings.
    """

    def __init__(self, encoding='onehot', categories='auto', dtype=np.float64,
                 handle_unknown='error'):
        self.encoding = encoding
        self.categories = categories
        self.dtype = dtype
        self.handle_unknown = handle_unknown

    def fit(self, X, y=None):
        """Fit the CategoricalEncoder to X.
        Parameters
        ----------
        X : array-like, shape [n_samples, n_feature]
            The data to determine the categories of each feature.
        Returns
        -------
        self
        """

        if self.encoding not in ['onehot', 'onehot-dense', 'ordinal']:
            template = ("encoding should be either 'onehot', 'onehot-dense' "
                        "or 'ordinal', got %s")
            raise ValueError(template % self.handle_unknown)

        if self.handle_unknown not in ['error', 'ignore']:
            template = ("handle_unknown should be either 'error' or "
                        "'ignore', got %s")
            raise ValueError(template % self.handle_unknown)

        if self.encoding == 'ordinal' and self.handle_unknown == 'ignore':
            raise ValueError("handle_unknown='ignore' is not supported for"
                             " encoding='ordinal'")

        X = check_array(X, dtype=np.object, accept_sparse='csc', copy=True)
        n_samples, n_features = X.shape

        self._label_encoders_ = [LabelEncoder() for _ in range(n_features)]

        for i in range(n_features):
            le = self._label_encoders_[i]
            Xi = X[:, i]
            if self.categories == 'auto':
                le.fit(Xi)
            else:
                valid_mask = np.in1d(Xi, self.categories[i])
                if not np.all(valid_mask):
                    if self.handle_unknown == 'error':
                        diff = np.unique(Xi[~valid_mask])
                        msg = ("Found unknown categories {0} in column {1}"
                               " during fit".format(diff, i))
                        raise ValueError(msg)
                le.classes_ = np.array(np.sort(self.categories[i]))

        self.categories_ = [le.classes_ for le in self._label_encoders_]

        return self

    def transform(self, X):
        """Transform X using one-hot encoding.
        Parameters
        ----------
        X : array-like, shape [n_samples, n_features]
            The data to encode.
        Returns
        -------
        X_out : sparse matrix or a 2-d array
            Transformed input.
        """
        X = check_array(X, accept_sparse='csc', dtype=np.object, copy=True)
        n_samples, n_features = X.shape
        X_int = np.zeros_like(X, dtype=np.int)
        X_mask = np.ones_like(X, dtype=np.bool)

        for i in range(n_features):
            valid_mask = np.in1d(X[:, i], self.categories_[i])

            if not np.all(valid_mask):
                if self.handle_unknown == 'error':
                    diff = np.unique(X[~valid_mask, i])
                    msg = ("Found unknown categories {0} in column {1}"
                           " during transform".format(diff, i))
                    raise ValueError(msg)
                else:
                    # Set the problematic rows to an acceptable value and
                    # continue `The rows are marked `X_mask` and will be
                    # removed later.
                    X_mask[:, i] = valid_mask
                    X[:, i][~valid_mask] = self.categories_[i][0]
            X_int[:, i] = self._label_encoders_[i].transform(X[:, i])

        if self.encoding == 'ordinal':
            return X_int.astype(self.dtype, copy=False)

        mask = X_mask.ravel()
        n_values = [cats.shape[0] for cats in self.categories_]
        n_values = np.array([0] + n_values)
        indices = np.cumsum(n_values)

        column_indices = (X_int + indices[:-1]).ravel()[mask]
        row_indices = np.repeat(np.arange(n_samples, dtype=np.int32),
                                n_features)[mask]
        data = np.ones(n_samples * n_features)[mask]

        out = sparse.csc_matrix((data, (row_indices, column_indices)),
                                shape=(n_samples, indices[-1]),
                                dtype=self.dtype).tocsr()
        if self.encoding == 'onehot-dense':
            return out.toarray()
        else:
            return out

# To here


# from sklearn.preprocessing import CategoricalEncoder # in future versions of Scikit-Learn

cat_encoder = CategoricalEncoder()
housing_cat_reshaped = housing_cat.values.reshape(-1, 1)
housing_cat_1hot = cat_encoder.fit_transform(housing_cat_reshaped)
housing_cat_1hot

# Step 4: write custom transformers

from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y = None):
        return self # nothing else to do
    def transform(self, X, y = None):
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room = False)
housing_extra_attributes = attr_adder.transform(housing.values)

# Step 5: Normalize (min-max scaling) all non-categorical data

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import FeatureUnion

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y = None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

from sklearn.pipeline import FeatureUnion

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

num_pipeline = Pipeline([
                         ('selector', DataFrameSelector(num_attribs)),
                         ('imputer', Imputer(strategy = "median")),
                         ('attribs_adder', CombinedAttributesAdder()),
                         ('std_scaler', MinMaxScaler()),
                        ])

cat_pipeline = Pipeline([('selector', DataFrameSelector(cat_attribs)), 
                         ('cat_encoder', CategoricalEncoder(encoding="onehot-dense")),
                        ])

full_pipeline = FeatureUnion(transformer_list = [("num_pipeline", num_pipeline), 
                                                 ("cat_pipeline", cat_pipeline),
                                                ])

# And we can now run the whole pipeline simply:

housing_prepared = full_pipeline.fit_transform(housing)
print(housing_prepared)
print(housing_prepared.shape)

# Each subpipeline starts with a selector transformer: it simply transforms the data by selecting the desired 
# attributes (numerical or categorical), dropping the rest, and converting the resulting DataFrame to NumPy
# array. There is nothing in Scikit-Learn to handle Pandas DataFrames, so we need to write a simple custom trans-
# former for this task:

from sklearn.base import BaseEstimator, TransformerMixin


class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values
    
# Step 6: Perform Lasso-based regression using a variety of values for  α  between 0 and 1 via a grid search 
# where housing_labels is the output and all other features are the input (similar to as seen in lecture two.)

# Create a Lasso-based regression

from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

lasso_reg = Lasso()
lasso_reg.fit(housing_prepared, housing_labels)

housing_predictions_lasso_reg = lasso_reg.predict(housing_prepared)
lasso_reg_mse = mean_squared_error(housing_labels, housing_predictions_lasso_reg)
lasso_reg_rmse = np.sqrt(lasso_reg_mse)
lasso_reg_rmse

# Cross-validate the Lasso

from sklearn.model_selection import cross_val_score

lasso_reg_scores = cross_val_score(lasso_reg, housing_prepared, housing_labels, scoring = "neg_mean_squared_error", 
                                cv = 10)

lasso_reg_rmse_scores = np.sqrt(-lasso_reg_scores)

# Create the display_scores function
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(lasso_reg_rmse_scores)

# Implement GridSearch to find the optimal alpha

from sklearn.model_selection import GridSearchCV

alphas = [i/100 for i in range(0, 101, 1)] # Do not delete! Originial!

param_grid = [{'alpha': alphas}] # Do not delete! Original!

lasso_reg = Lasso()

grid_search = GridSearchCV(lasso_reg, param_grid, cv=5, scoring='neg_mean_squared_error', refit = True)

grid_search.fit(housing_prepared, housing_labels)

print(grid_search.best_params_)

print(grid_search.best_estimator_)

# Most optimal output/Lasso regression model is the following: 

# {'alpha': 1.0}
# Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
#    normalize=False, positive=False, precompute=False, random_state=None,
#    selection='cyclic', tol=0.0001, warm_start=False)

# Now let's get the evaluation score:

cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)
    
# We get the following output for:

# param_grid = [{'alpha': alphas}] 

"""
68530.40517791889 {'alpha': 0.0}
68530.21647315132 {'alpha': 0.01}
68530.02799662169 {'alpha': 0.02}
68529.85317351531 {'alpha': 0.03}
68529.66963344108 {'alpha': 0.04}
68529.48632405429 {'alpha': 0.05}
68529.3032453643 {'alpha': 0.06}
68529.12039737291 {'alpha': 0.07}
68528.93778008201 {'alpha': 0.08}
68528.75539349343 {'alpha': 0.09}
68528.573237609 {'alpha': 0.1}
68528.39131243058 {'alpha': 0.11}
68528.20961795999 {'alpha': 0.12}
68528.02815419907 {'alpha': 0.13}
68527.84692114966 {'alpha': 0.14}
68527.66591881357 {'alpha': 0.15}
68527.48514719265 {'alpha': 0.16}
68527.30460628873 {'alpha': 0.17}
68527.1242961036 {'alpha': 0.18}
68526.94421663911 {'alpha': 0.19}
68526.7643678971 {'alpha': 0.2}
68526.58474987933 {'alpha': 0.21}
68526.40536258765 {'alpha': 0.22}
68526.22620602387 {'alpha': 0.23}
68526.0472801898 {'alpha': 0.24}
68525.86858508723 {'alpha': 0.25}
68525.69012071798 {'alpha': 0.26}
68525.51188708385 {'alpha': 0.27}
68525.33388418664 {'alpha': 0.28}
68525.15611202814 {'alpha': 0.29}
68524.97857061018 {'alpha': 0.3}
68524.8012599345 {'alpha': 0.31}
68524.62418000294 {'alpha': 0.32}
68524.44733081726 {'alpha': 0.33}
68524.27071237924 {'alpha': 0.34}
68524.0943246907 {'alpha': 0.35}
68523.91816775339 {'alpha': 0.36}
68523.7422415691 {'alpha': 0.37}
68523.5665461396 {'alpha': 0.38}
68523.39108146669 {'alpha': 0.39}
68523.21584755211 {'alpha': 0.4}
68523.04084439765 {'alpha': 0.41}
68522.86607200507 {'alpha': 0.42}
68522.69153037615 {'alpha': 0.43}
68522.51721951264 {'alpha': 0.44}
68522.3431394163 {'alpha': 0.45}
68522.16929008889 {'alpha': 0.46}
68521.99567153218 {'alpha': 0.47}
68521.8222837479 {'alpha': 0.48}
68521.64912673783 {'alpha': 0.49}
68521.47620050369 {'alpha': 0.5}
68521.30350504724 {'alpha': 0.51}
68521.13104037024 {'alpha': 0.52}
68520.9588064744 {'alpha': 0.53}
68520.7868033615 {'alpha': 0.54}
68520.61503103325 {'alpha': 0.55}
68520.44348949139 {'alpha': 0.56}
68520.27217873765 {'alpha': 0.57}
68520.10109877378 {'alpha': 0.58}
68519.93024960149 {'alpha': 0.59}
68519.7596312225 {'alpha': 0.6}
68519.58924363858 {'alpha': 0.61}
68519.4190868514 {'alpha': 0.62}
68519.2491608627 {'alpha': 0.63}
68519.07946567418 {'alpha': 0.64}
68518.91000128759 {'alpha': 0.65}
68518.74076770461 {'alpha': 0.66}
68518.57176492696 {'alpha': 0.67}
68518.40299295635 {'alpha': 0.68}
68518.23445179449 {'alpha': 0.69}
68518.0661414431 {'alpha': 0.7}
68517.89806190383 {'alpha': 0.71}
68517.73021317842 {'alpha': 0.72}
68517.56259526855 {'alpha': 0.73}
68517.39520817592 {'alpha': 0.74}
68517.22805190223 {'alpha': 0.75}
68517.06112644915 {'alpha': 0.76}
68516.89443181838 {'alpha': 0.77}
68516.7279680116 {'alpha': 0.78}
68516.5617350305 {'alpha': 0.79}
68516.39573287676 {'alpha': 0.8}
68516.22996155203 {'alpha': 0.81}
68516.06442105802 {'alpha': 0.82}
68515.89911139639 {'alpha': 0.83}
68515.73403256881 {'alpha': 0.84}
68515.56918457695 {'alpha': 0.85}
68515.40456742246 {'alpha': 0.86}
68515.24018110704 {'alpha': 0.87}
68515.07602563233 {'alpha': 0.88}
68514.912101 {'alpha': 0.89}
68514.74840721168 {'alpha': 0.9}
68514.58067429079 {'alpha': 0.91}
68514.36706202409 {'alpha': 0.92}
68514.19708604846 {'alpha': 0.93}
68514.03170227849 {'alpha': 0.94}
68513.86785668322 {'alpha': 0.95}
68513.70479805618 {'alpha': 0.96}
68513.54222320885 {'alpha': 0.97}
68513.34751052012 {'alpha': 0.98}
68513.18388303602 {'alpha': 0.99}
68513.02130694577 {'alpha': 1.0}
"""

# In this example, we obtain the best solution by setting alpha = 1. The RMSE score for this combination is 68513.02130694577

# Step 7: Evaluate your system on the test set

final_model = grid_search.best_estimator_

X_test = strat_test_set.drop("median_house_value", axis = 1)
y_test = strat_test_set["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test)

final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

print(final_rmse) # we get 66808.43524898415

[[0.24501992 0.50478215 0.7254902  ... 0.         0.         0.        ]
 [0.24103586 0.47927736 0.25490196 ... 0.         0.         0.        ]
 [0.71215139 0.02444208 0.58823529 ... 0.         0.         1.        ]
 ...
 [0.79183267 0.16471838 0.15686275 ... 0.         0.         0.        ]
 [0.6314741  0.1360255  0.58823529 ... 0.         0.         0.        ]
 [0.18924303 0.55579171 1.         ... 0.         1.         0.        ]]
(16512, 19)


/home/isaac/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/isaac/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/isaac/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/isaac/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not conver

Scores: [66058.10675541 66772.69218509 67677.33037409 74624.95914437
 68037.27357579 70909.87996377 64178.25642625 68082.51123534
 70939.94829147 67303.25171369]
Mean: 68458.42096652606
Standard deviation: 2817.4458694685127


/home/isaac/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/isaac/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
/home/isaac/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:477: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
/home/isaac/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha ma

/home/isaac/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/isaac/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/isaac/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/isaac/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not conver

...

/home/isaac/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/isaac/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/isaac/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/isaac/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not conver

/home/isaac/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/isaac/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/isaac/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/isaac/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not conver

/home/isaac/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/isaac/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/isaac/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/isaac/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not conver

{'alpha': 1.0}
Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)
68530.40517791889 {'alpha': 0.0}
68530.21647315132 {'alpha': 0.01}
68530.02799662169 {'alpha': 0.02}
68529.85317351531 {'alpha': 0.03}
68529.66963344108 {'alpha': 0.04}
68529.48632405429 {'alpha': 0.05}
68529.3032453643 {'alpha': 0.06}
68529.12039737291 {'alpha': 0.07}
68528.93778008201 {'alpha': 0.08}
68528.75539349343 {'alpha': 0.09}
68528.573237609 {'alpha': 0.1}
68528.39131243058 {'alpha': 0.11}
68528.20961795999 {'alpha': 0.12}
68528.02815419907 {'alpha': 0.13}
68527.84692114966 {'alpha': 0.14}
68527.66591881357 {'alpha': 0.15}
68527.48514719265 {'alpha': 0.16}
68527.30460628873 {'alpha': 0.17}
68527.1242961036 {'alpha': 0.18}
68526.94421663911 {'alpha': 0.19}
68526.7643678971 {'alpha': 0.2}
68526.58474987933 {'alpha': 0.21}
68526.40536258765 {'alpha': 0.22}
68526.22620602387 {'

/home/isaac/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)



Why is it necessary to normalize all continuous variables before performing Lasso? (OPTIONAL)

<pre> One the reasons why it is necessary to normalize all continuous variables before performing Lasso is that the input numerical attributes have very different scales. Therefore, normlization ensures that:
1) the training set is less sensitive to the scale of features,
2) regularization behaves similarly across the training set. For example, if we did not normalize the data, regularization will have greater effect on data with greater weights and smaller effects on data with smaller weights.
3) comparison of results across models is consistent. 
4) optimization is well-conditioned.
</pre>

### Conclusions
For what values of $\alpha$ does Lasso perform best? Does it perform as well on the housing data as the linear regressor from the lectures? Why do you think this is?

<pre> Lassso performs best when the value of alpha is 1 as we can see from the training set's RMSE of 68513.02130694577. It performs a bit better than the linear regression (the value of RMSE is 68628.413493824875). One of the reasons why the performance of Lasso Regression may be similar to that of Linear Regression is that for the small values of alpha the Lasso Regression resembles the Regression Regression due to the penalty (alpha * L1 norm of the weights) being small. Therefore, Lasso Regression fits training data better than the Linear Regression. On the other hand, when it comes to the test set, Lasso Regression has an RMSE of 66808.43524898415 as compared to 48209.6 of the Linear Regression. Therefore, Lasso Regression underfits the the test data as compared to the Linear Regression. It seems that the penalty f-n has a greater negative effect on the test data. From my understanding, this is due to the effect of 'sparsity' - as we increase the value of alpha, the amount of features with the zero coefficient increases thus leading to the underfitting.</pre>

### Read appending B

- Reflect on your last data project, read appendix B. Then, write down a few of the checklist items that your last data project could have used. If you have not yet done a data project, then write down a few of the items that you found most interesting.


In [90]:
# According to the Appendix B, there are 8 main steps to the ML project:
# 1. Frame the problem and look at the big picture.
# 2. Get the data.
# 3. Explore the data to gain insights.
# 4. Prepare the data to better expose the underlying data patterns to Machine Learn‐
# ing algorithms.
# 5. Explore many different models and short-list the best ones.
# 6. Fine-tune your models and combine them into a great solution.
# 7. Present your solution.
# 8. Launch, monitor, and maintain your system.
# Now, let's compare it what we have done in our project with the Lasso Regression:
# 1. Given the problem, I would like to have learned the following in regards to the housing dataset:
# - How to define the objective in business terms?
# - How will our solution be used?
# - Is the performance measure aligned with the business objective?
# - What would be the minimum performance measure needed to reach the business objective?
# 2. Covered in the book.
# 3. Covered in the book.
# 4. Covered in the book.
# 5. Covered in the book. Would like to learn more about how to apply other regression models such as Logistic
# Regression, Bayesian Regression, and Ridge Regression.
# 6. Covered in the book.
# 7. It would be interesting to make a mini presentation in regards to the project and explain why our solution
# achieves the business objective.
# 8. Lastly, it would be great to see our project to go into some sort of production to see how code is implemented
# in the industry. That way we would be able to learn how to write monitoring code to check our system's live per-
# formance and retrain our model on a regular basis on fresh data.